In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import DataFrameLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma



In [ ]:

# Load environment variables from .env file
import os
from dotenv import load_dotenv
load_dotenv()

# Load the OpenAI API key from environment variables   
openai_api_key = os.getenv("OPENAI_API_KEY")


In [ ]:

# Load your data
df = pd.read_csv("../data/data_csv/ADASpreadsheet.csv")
clean_df = df.fillna("See section note")
df_loader = DataFrameLoader(clean_df, page_content_column="Subsection_Note")
documents = df_loader.load()

In [6]:
embeddings = OpenAIEmbeddings(api_key=openai_api_key)


C:\Users\elcoo\AppData\Local\Temp\ipykernel_25324\4028280539.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(api_key=openai_api_key)


In [7]:
vector_store = Chroma.from_documents(documents, embedding=embeddings, persist_directory='db')

In [8]:

llm = ChatOpenAI(api_key=openai_api_key, model='gpt-4', temperature=0.7)
retriever = vector_store.as_retriever(search_kwargs={"k":3})
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"  # Specify the output key for memory storage
)


C:\Users\elcoo\AppData\Local\Temp\ipykernel_25324\2727320299.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(api_key=openai_api_key, model='gpt-4', temperature=0.7)
C:\Users\elcoo\AppData\Local\Temp\ipykernel_25324\2727320299.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [9]:

crc = ConversationalRetrievalChain.from_llm(
    llm,
    retriever,
    memory=memory,
    return_source_documents=True,
    output_key="answer"  # Specify the output key for the chain
)



In [ ]:
# Function to handle chat interactions
def chat_with_documents():
    print("Welcome to the Document Chatbot! Type 'exit' to end the conversation.")
    
    while True:
        user_query = input("\nYour question: ")
        
        if user_query.lower() == 'exit':
            print("Thank you for using the Document Chatbot. Goodbye!")
            break
        
        # Get response from the chain
        response = crc.invoke({"question": user_query})
        
        # Print the response
        print("\nChatbot:", response["answer"])

# Run the chatbot
if __name__ == "__main__":
    chat_with_documents()